In [1]:
from hgb_maternal_disorder_paf_generation import *
import matplotlib.pyplot as plt

In [2]:
import warnings

warnings.filterwarnings('ignore')

In [3]:
draw = 'draw_0'
location = 'pakistan'

In [4]:
results_directory = os.getcwd() + '/'

In [5]:
get_simulated_population(location, draw)

2025-09-20 09:24:08.126 | INFO     | simulation_1-artifact_manager:79 - Running simulation from artifact located at /mnt/team/simulation_science/pub/models/vivarium_gates_mncnh/artifacts/model16.0/pakistan.hdf.
2025-09-20 09:24:08.128 | INFO     | simulation_1-artifact_manager:80 - Artifact base filter terms are ['draw == 0'].
2025-09-20 09:24:08.129 | INFO     | simulation_1-artifact_manager:81 - Artifact additional filter terms are None.
2025-09-20 09:24:08.128 | INFO     | simulation_1-artifact_manager:80 - Artifact base filter terms are ['draw == 0'].
2025-09-20 09:24:08.129 | INFO     | simulation_1-artifact_manager:81 - Artifact additional filter terms are None.
2025-09-20 09:24:32.550 | WARNING  | simulation_1-results_manager:446 - Specified excluded stratifications are already not included by default: ['stillbirth', 'partial_term']
2025-09-20 09:24:32.551 | WARNING  | simulation_1-results_manager:446 - Specified excluded stratifications are already not included by default: ['st

(        alive     sex        age  hemoglobin_exposure
 0       alive  Female  27.686990           100.835463
 1       alive  Female  16.901275           116.681960
 2       alive  Female  25.907499           103.367641
 3       alive  Female  27.566019           117.575350
 4       alive  Female  30.891936           113.869808
 ...       ...     ...        ...                  ...
 199995  alive  Female  21.809742            90.293515
 199996  alive  Female  35.677560           120.137529
 199997  alive  Female  23.777497           117.214950
 199998  alive  Female  25.422869           120.917007
 199999  alive  Female  30.762903           103.196673
 
 [200000 rows x 4 columns],
 InteractiveContext())

In [6]:
load_maternal_disorders(location, draw)

2025-09-20 09:24:48.716 | INFO     | simulation_2-artifact_manager:79 - Running simulation from artifact located at /mnt/team/simulation_science/pub/models/vivarium_gates_mncnh/artifacts/model16.0/pakistan.hdf.
2025-09-20 09:24:48.717 | INFO     | simulation_2-artifact_manager:80 - Artifact base filter terms are ['draw == 0'].
2025-09-20 09:24:48.718 | INFO     | simulation_2-artifact_manager:81 - Artifact additional filter terms are None.


2025-09-20 09:25:10.997 | WARNING  | simulation_2-results_manager:446 - Specified excluded stratifications are already not included by default: ['stillbirth', 'partial_term']
2025-09-20 09:25:10.998 | WARNING  | simulation_2-results_manager:446 - Specified excluded stratifications are already not included by default: ['stillbirth', 'partial_term']
2025-09-20 09:25:11.258 | INFO     | simulation_2-results_manager:134 - The following stratifications are registered but not used by any observers: 
['sex']


,alive,sex,age,hemoglobin_exposure,hemoglobin.exposure,hemoglobin_on_maternal_hemorrhage.relative_risk,hemoglobin_on_maternal_sepsis_and_other_maternal_infections.relative_risk,maternal_hemorrhage.incidence_risk,hemorrhage_paf,maternal_sepsis_and_other_maternal_infections.incidence_risk,sepsis_paf,age_group,location
0,alive,Female,27.686990,100.835463,100.835463,1.554026,1.326311,0.171490,-0.109966,0.169817,-0.119372,25_to_29,pakistan
1,alive,Female,16.901275,116.681960,116.681960,1.023450,1.033366,0.197232,-0.119853,0.647812,-0.114808,15_to_19,pakistan
2,alive,Female,25.907499,103.367641,103.367641,1.417562,1.270624,0.156431,-0.109966,0.162687,-0.119372,25_to_29,pakistan
3,alive,Female,27.566019,117.575350,117.575350,1.015586,1.023039,0.112072,-0.109966,0.130987,-0.119372,25_to_29,pakistan
4,alive,Female,30.891936,113.869808,113.869808,1.060334,1.071647,0.155406,-0.111554,0.099824,-0.118662,30_to_34,pakistan
...,...,...,...,...,...,...,...,...,...,...,...,...,...
199995,alive,Female,21.809742,90.293515,90.293515,2.291758,1.532377,0.254868,-0.106462,0.369809,-0.113970,20_to_24,pakistan
199996,alive,Female,35.677560,120.137529,120.137529,0.999359,0.998923,0.144454,-0.113822,0.077917,-0.116999,35_to_39,pakistan
199997,alive,Female,23.777497,117.214950,117.214950,1.018627,1.027091,0.113282,-0.106462,0.247868,-0.113970,20_to_24,pakistan
199998,alive,Female,25.422869,120.917007,120.917007,0.995857,0.993308,0.109895,-0.109966,0.127180,-0.119372,25_to_29,pakistan


In [7]:
calculate_pafs(location, draw)

2025-09-20 09:25:36.610 | INFO     | simulation_3-artifact_manager:79 - Running simulation from artifact located at /mnt/team/simulation_science/pub/models/vivarium_gates_mncnh/artifacts/model16.0/pakistan.hdf.
2025-09-20 09:25:36.611 | INFO     | simulation_3-artifact_manager:80 - Artifact base filter terms are ['draw == 0'].
2025-09-20 09:25:36.612 | INFO     | simulation_3-artifact_manager:81 - Artifact additional filter terms are None.
2025-09-20 09:25:36.611 | INFO     | simulation_3-artifact_manager:80 - Artifact base filter terms are ['draw == 0'].
2025-09-20 09:25:36.612 | INFO     | simulation_3-artifact_manager:81 - Artifact additional filter terms are None.


2025-09-20 09:25:59.036 | WARNING  | simulation_3-results_manager:446 - Specified excluded stratifications are already not included by default: ['stillbirth', 'partial_term']
2025-09-20 09:25:59.037 | WARNING  | simulation_3-results_manager:446 - Specified excluded stratifications are already not included by default: ['stillbirth', 'partial_term']
2025-09-20 09:25:59.037 | WARNING  | simulation_3-results_manager:446 - Specified excluded stratifications are already not included by default: ['stillbirth', 'partial_term']
2025-09-20 09:25:59.298 | INFO     | simulation_3-results_manager:134 - The following stratifications are registered but not used by any observers: 
['sex']
2025-09-20 09:25:59.298 | INFO     | simulation_3-results_manager:134 - The following stratifications are registered but not used by any observers: 
['sex']


(  age_group       paf  location
 0  10_to_14  0.359031  pakistan
 1  15_to_19  0.287729  pakistan
 2  20_to_24  0.245374  pakistan
 3  25_to_29  0.250555  pakistan
 4  30_to_34  0.245782  pakistan
 5  35_to_39  0.259342  pakistan
 6  40_to_44  0.268225  pakistan
 7  45_to_49  0.266679  pakistan
 8  50_to_54  0.306116  pakistan,
   age_group       paf  location
 0  10_to_14  0.198284  pakistan
 1  15_to_19  0.161155  pakistan
 2  20_to_24  0.146736  pakistan
 3  25_to_29  0.147361  pakistan
 4  30_to_34  0.144888  pakistan
 5  35_to_39  0.150419  pakistan
 6  40_to_44  0.156080  pakistan
 7  45_to_49  0.155915  pakistan
 8  50_to_54  0.168187  pakistan)

# Next steps
- Add depressive disorders to calculation (make model run request 16.5)
- Test different population sizes and plot PAFs (make population size a column in CSV, start at 20,000 and go up a factor of 10 5 times, keep age stratifications - fringe age groups expected to look funny, make note of approximate time it takes for each calculation)
- Do more testing to figure out if/why PAFs in artifact are so different than PAFs calculated in the interative sim here
- Run for all draws, locations, age groups, and outcomes and save new CSVs

In [8]:
# Pull hemoglobin PAFs from GBD for comparison
